In [22]:
# Import packages
import urllib.request
import time
import random
import pandas as pd


### Restuarant URLs with random 50 Pure-Live and 50 Non-Live Restaurants

In [149]:
# Read data for restaurant name and URLs (Input file)
restaurants = pd.read_excel("restaurants_2.xlsx")

In [150]:
# Store Restaurant URLs and names in lists
rest_links = restaurants['Restaurant_URL'].tolist()
rest_names = restaurants['Restaurant_Name'].tolist()
print(len(rest_links),len(rest_names))

100 100


#### Scrape reviews for 100 restaurants

In [32]:
for rest in range(len(rest_links)):
    # Create empty lists for review, author, rating and restaurant
    review = []
    author = []
    rating = []
    restaurant = []

    # Store Main URL
    yelp_restaurant_url = rest_links[rest]
    # Set sleep time
    time.sleep(3)

    # Open Main URL
    html = urllib.request.urlopen(yelp_restaurant_url).read().decode('utf-8')

    # Create a list to store all pages
    final_links = [yelp_restaurant_url]

    # Iterate over all next links and store them in a list 
    while html.find("<link rel=\"next\" href=\"") != -1:
        start_url = html.find("<link rel=\"next\" href=\"")
        end_url = html.find("<script type=\"application/ld+json\">")
        new_link = html[start_url+23:end_url-11]
        final_links.append(new_link)
        time.sleep(3)
        html = urllib.request.urlopen(new_link).read().decode('utf-8')

    # Iterate over all links
    for link in final_links:
        time.sleep(3)
        html = urllib.request.urlopen(link).read().decode('utf-8')
        b = html.find("{\"reviewRating\"")
        c = html.find("\"@context\"")
        html2 = html[b+1:c-2]

        # Iterate to store all reviews and rating
        while html2.find("\"reviewRating\"") != -1:
            desc_index = html2.find("\"description\"")
            author_index = html2.find("\"author\"")
            desc = html2[desc_index+16:author_index-3]
            desc = desc.replace("\\n\\n"," ")
            desc = desc.replace("\\n"," ")
            desc = desc.replace("\\u0026","&")
            review.append(desc)
            rating_index = html2.find("\"ratingValue\"")
            rate_end = html2.find("\"datePublished\"")
            r = html2[rating_index+15:rate_end-3]
            rating.append(r)

            remaining = html2[author_index:] 
            author_index = remaining.find("\"author\"")
            if remaining.find("\"reviewRating\"") != -1:
                end = remaining.find("\"reviewRating\"")
                author.append(remaining[author_index+11:end-5])
            else:
                end = remaining.find("}]")
                author.append(remaining[author_index+11:end-1])
            html2 = remaining[end:]

    # Store Restaurant name in a list
    for i in range(len(review)):
        restaurant.append(rest_names[rest])

    # Write each restaurant data to csv and merge all during sentiment analysis    
    outputdf = pd.DataFrame(list(zip(restaurant,rating, review)),columns=["Name", "Rating","Review"])
    file_name = rest_names[rest] + '.csv'
    outputdf.to_csv(file_name, index = False)